#### Setup

In [2]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)

3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [3]:
# GLOBAL VARIABLES
field_width = 100
field_height = 100
depot_x = 50
depot_y = 50

In [71]:
class Instance():
    
    def __init__(self, xlocs, ylocs, demands):

        self.size = len(demands)-1
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.distances = self.calc_distance_matrix()
        self.tour = 'None'
        
    def calc_distance_matrix(self):

        distances = np.zeros((self.size+1, self.size+1), dtype=float)
        for i in range(self.size+1):
            for j in range(self.size+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def get_lowerbound(self, capacity):
        return (2/capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])
    
    def get_fleet_size(self, route_size):
        assert self.size % route_size == 0, "Number of customers must be evenly divisible by the route size."
        return int(self.size / route_size)
    
    def solve_TSP(self):
        solver = TSPSolver.from_data(inst.xlocs, inst.ylocs, 'EUC_2D')
        solution = solver.solve()
        self.tour = solution.tour

In [82]:
def solve_TSP(inst):
    
    solver = TSPSolver.from_data(inst.xlocs, inst.ylocs, 'EUC_2D')
    solution = solver.solve()
    return solution

def get_dedicated_routes(inst, route_size):
    
    tour =  inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size]
        routes.append(new_route)
    return routes


In [7]:
def get_circular_cost(inst,segment):
    return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Assumes vehicle travels to/from the depot at segment endpoints."""
    return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

In [ ]:
def solve_fully_flexible(inst):
    
    # Get pairwise distances
    nodes = {}
    nodes.update(depots)
    nodes.update(customers)
    distances = calc_dist_matrix(nodes)
    
    # Keep customers with non-zero demand
    nonzero_tau = dict([(i, tau[i]) for i in tau if tau[i].d >0])
    
    # Use bin-packing to determine  route endpoints (list indices)
    demands = [nonzero_tau[i].d for i in nonzero_tau]
    if len(demands) > 0:
        endpoints = [0] + get_breakpoint_indices(demands, Q) + [len(demands)-1]
        assert len(endpoints) % 2 == 0, "Assertion Error: There is not an even number of endpoints."
        num_trips = len(endpoints)/2
    else:
        return (0,0,0,0) # No demands to fill, so no transportation costs or trips
    
    # Use end points to create Routes and calculate costs
    greedy_routes = {}
    circular_costs = []
    radial_costs = []
    total_costs = []
    
    j = 1
    for e in range(0,len(endpoints)-1,2):
        
        # Create new route
        start = endpoints[e] # Position of first customer
        end = endpoints[e+1] # Position of last customer
        sub_seq = dict([(i, nonzero_tau[i]) for i in nonzero_tau if i >= [*nonzero_tau][start] and i <= [*nonzero_tau][end]])
        greedy_routes[j] = Route(sub_seq, j)
        
        # TO-DO: Need to update to accomodate split demand

        # Calculate radial costs at beginning and end of route
        radial = distances[depot.ID, greedy_routes[j].customers[0].ID] + distances[depot.ID, greedy_routes[j].customers[-1].ID]
        radial_costs.append(radial)

        # Calculate circular cost
        route_edges = get_edge_list(greedy_routes[j],depot) # Edge list
        circular = sum([distances[e] for e in route_edges])
        circular -= radial # TEMP FIX: remove radial cost from circular to prevent double counting
        circular_costs.append(circular)
        
        # Total cost
        total_costs.append(circular+radial)
        j+=1
    
    """
    ### Generate report ###
    
    print('N=%s, M=%s, Q=%s \n' %(len(customers),len(greedy_routes),Q))
 
    for j in greedy_routes:
        print("Truck %s Route (pos, d) -->" %j, [(list(greedy_routes[j].sequence.keys())[i],
                                                 greedy_routes[j].customers[i].ID,
                                                 greedy_routes[j].customers[i].d) for i in range(greedy_routes[j].n)])
    print('')
    print('Starting Position (s), Workload (w), Excess Capacity (e), Final Position (f):')
    for j in greedy_routes:
        print('Route %s: s=%s, w=%s, e=%s, f=%s'
              %(j,[*greedy_routes[j].sequence][0],greedy_routes[j].d, Q-greedy_routes[j].d, [*greedy_routes[j].sequence][-1]))
    print('')
    print('Circular Costs --> ', np.round(circular_costs))
    print('Radial Costs --> ', np.round(radial_costs))
    print('Total Costs --> ', np.round(np.add(radial_costs,circular_costs)))

    """

    
    return sum(circular_costs), sum(radial_costs), sum(total_costs), num_trips

In [84]:
N = 10
dmin = 0
dmax = 8
cust_x = field_width*np.random.random(N)
cust_y = field_height*np.random.random(N)
cust_dems = np.random.randint(dmin,dmax,N)
xlocs = np.append([depot_x], cust_x)
ylocs = np.append([depot_y], cust_y)
demands = np.append([0], cust_dems)
inst = Instance(xlocs, ylocs, demands)

In [53]:
capacity=20
inst.get_lowerbound(capacity)

158.970585216701

In [55]:
route_size = 5
inst.get_fleet_size(route_size)

2

In [73]:
inst.tour

'None'

In [86]:
inst.solve_TSP()
inst.tour

array([ 0,  5,  6,  4,  1,  9,  8,  7,  2, 10,  3], dtype=int32)

In [87]:
#tsp = solve_TSP(inst)
inst.solve_TSP()
routes = get_dedicated_routes(inst, route_size)
print('Big tour:', inst.tour)
print('Routes:', routes)
print('Radial costs:', [get_radial_cost(inst,seg).round(1) for seg in routes])
print('Circular costs:', [get_circular_cost(inst,seg).round(1) for seg in routes])

Big tour: [ 0  5  6  4  1  9  8  7  2 10  3]
Routes: [array([5, 6, 4, 1, 9], dtype=int32), array([ 8,  7,  2, 10,  3], dtype=int32)]
Radial costs: [69.3, 54.5]
Circular costs: [42.8, 93.9]
